## Transferencia de aprendizaje 
Utilizar conceptos de un problema en otro

En redes convolucionales, se utilizan los pesos de la parte de extracción características o feature extraction, correspondientes con las capas convolucionales y de pooling.

![](img/cnn.jpeg)

### Problema a resolver: Clasificación binaria de imágenes de fondo de ojo para detectar retinopatía diabética

#### 5 niveles de severidad repartidos en 2 categorías para realizar una clasificación binaria
![](images/image1.png)

Existen redes preentrenadas en la base de datos **ImageNet**, las cuales pueden ser cargadas desde Keras


  
![](img/imagenet.jpeg)
<img src="img/networks.png" width="700">

La base de datos usada es la de **Kaggle: Diabetic retinopathy detection**
<img src="img/histogram.png" width="500">


Antes de resolver el problema con todos los niveles de severidad de la enfermedad, resolvemos un problema **más simple**:
![](img/image6.png)

Y utilizamos métodos más simples primero, como **Random Forest, SVM, MLP y KNN**
<img src="img/metodos.png" width="300">


Como no funcionaron usamos CNN en el problema simple de **solo 2 niveles** para ver si funcionan las redes.
Vemos primero las tasas de aprendizajes y la constante de regularización apropiadas
<img src="img/lr.png" width="700">


Luego vemos si los métodos de regularización son necesarios
<img src="img/regularizacion.png" width="700">


Hasta que profundidad entrenar?
## Red Inception V3
<img src="img/arquitectura.png" width="700">

Fine tuning hasta varias profundidades
<img src="img/profundidad.png" width="700">

Ahora que tenemos idea del valor de todos los hiperparámetros, podemos intentar resolver el problema completo

<img src="img/image1.png" width="700">

Se obtuvieron las siguiente soluciones con imágenes de 224x224 
<img src="img/resultados1.png" width="400">


Se puede visualizar las regiones de importancia a la hora de clasificar? 

**Método de mapas de activación de clases**: solo funciona para redes de arquitectura particular
<img src="img/CAM.png" width="700">

**Método Guided Grad CAM:** Método para arquitecturas más custom y además da pixeles importantes
<img src="img/Guided_cam.png" width="700">

Se aplicaron los métodos **CAM** y **Guided Grad CAM** a las soluciones obtenidas
<img src="img/Xception_224.png" width="1100">


Se aplicaron los métodos **CAM** y **Guided Grad CAM** a las soluciones obtenidas

<img src="img/Densenet_224.png" width="1100">

Qué pasa si entrenamos redes con tamaños de imagen más grande?
<img src="img/solucion2.png" width="600">
La precisión mejora en todos los casos, por qué?

Aplicamos los métodos **CAM** y **Guided grad CAM** a las soluciones con imágenes más grandes
<img src="img/Xception_448.png" width="1100">

Aplicamos los métodos **CAM** y **Guided grad CAM** a las soluciones con imágenes más grandes
<img src="img/DenseNet_448.png" width="1100">

Tomamos una imágen que antes era clasificada mal y ahora bien, y le aplicamos los métodos de visualización

### DenseNet 224x224
<img src="img/not_ok.png" width="1000">

### DenseNet 448x448
<img src="img/red_ok.png" width="1000">
